In [2]:
import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
sns.set_context('notebook')
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
#rcParams['font.family'] = 'sans-serif'
#rcParams['font.sans-serif'] = ['Tahoma']
import tqdm
### notebook specific configuration ###
from os.path import basename
mpl.style.use('fivethirtyeight')

from Modules.Homer import *

sys.path.append('/home/isshamie/software/NGS-Pipeline')

data_folder = '/data/isshamie/CH_tissue_TSS/'

ref_fa = '/data/genome/hamster/picr/picr.fa'
annotation = '/data/genome/hamster/picr/updated_final_sort.gff3'
tss_annotation = '/data/isshamie/genome/start_site_mRNA_updated_final_sort.tsv'
mrna_peak = '/data/isshamie/TSS_CHO/mRNA.peak'

tissues = ['BloondnegSpleen','BMDM1hKLA1','BMDMwt','Brain','Brain7neg1',
          'FemaleReproductive','Heart','Intestine','Kidney','Liver','Lung',
           'MiscOrgans','Muscle','Pancreas','Skin','Spleen','Total','CHO']

len(tissues)

### Load in meta_sampe

meta_samples = pd.read_csv('Results/meta_samples.csv',index_col=0)

long_meta_samples = pd.read_csv('Results/long_meta_samples.csv',index_col=0)


/home/isshamie/software/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
secM_genes = pd.read_csv('/data/isshamie/CH_tissue_TSS/Supplemental_Data/amir.low.exp.csv',index_col=0)
secM_genes.head()

,humanID,choID,mean.read.count,is.secM.Amir
humanSymbol,,,,
A3GALT2,127550,100774283,0.114583,True
ABO,28,100772592,0.437500,True
ABO,28,100754666,0.437500,True
ABO,28,100758032,0.437500,True
AGAP2,116986,100764978,0.875000,True


In [4]:
anno_peaks = pd.read_csv('Results/peaks_with_tss.tsv',sep='\t',index_col=0)
anno_peaks = anno_peaks[['Chr','Start','End','Strand','Peak Score','Nearest TSS','isSameStrand','Distance to TSS']]
anno_peaks = anno_peaks[~anno_peaks['Nearest TSS'].isnull()]

In [5]:
anno_peaks.head()

,Chr,Start,End,Strand,Peak Score,Nearest TSS,isSameStrand,Distance to TSS
PeakID (cmd=annotatePeaks.pl Results/merged_samples/duplicatesSeparate_peaks_merged.tsv /data/genome/hamster/picr/picr.fa -gff /data/genome/hamster/picr/updated_final_sort.gff3),,,,,,,,
Merged-picr_33-4827738-24,picr_33,4827637,4827815,+,277526.625000,asmbl_45526,True,4.0
Merged-picr_117-401925-25,picr_117,401850,402000,-,225524.000000,asmbl_11728,True,-147.0
Merged-picr_35-689698-12,picr_35,689623,689773,-,199731.640625,asmbl_46769,False,1404.0
Merged-picr_76-305376-6,picr_76,305301,305451,-,184552.890625,asmbl_75726,False,-23522.0
Merged-picr_54-9283439-16,picr_54,9283364,9283514,+,146467.000000,asmbl_62508,False,-871.0


In [17]:
anno = pd.read_csv(annotation,sep='\t',header=None)

In [18]:
anno.head()

,0,1,2,3,4,5,6,7,8
0,picr_0,merge,gene,10,41421,.,+,.,ID=gene_1_1;gene_id=gene_1_1;gene=gene_1
1,picr_0,merge,mRNA,10,41421,.,+,.,ID=asmbl_1;Parent=gene_1_1;gene_id=gene_1_1;ge...
2,picr_0,merge,exon,10,4041,.,+,.,ID=asmbl_1.exon1;Parent=asmbl_1;gene_id=gene_1...
3,picr_0,merge,gene,176,38694,.,+,.,ID=gene_1_2;gene_id=gene_1_2;gene=SERHL
4,picr_0,merge,mRNA,176,38694,.,+,.,ID=asmbl_2;Parent=gene_1_2;gene_id=gene_1_2;ge...


In [19]:
anno.loc[0,8]

'ID=gene_1_1;gene_id=gene_1_1;gene=gene_1'

In [26]:
for i in range(10):
    print mrna_anno.loc[i,8]

ID=asmbl_1;Parent=gene_1_1;gene_id=gene_1_1;gene=gene_1;transcript_id=asmbl_1;cs=5
ID=asmbl_2;Parent=gene_1_2;gene_id=gene_1_2;gene=SERHL;transcript_id=asmbl_2;cs=2
ID=asmbl_3;Parent=gene_1_3;gene_id=gene_1_3;gene=gene_1;transcript_id=asmbl_3;cs=5
ID=asmbl_4;Parent=gene_2;gene_id=gene_2;gene=RRP7A;transcript_id=asmbl_4;cs=1
ID=asmbl_5;Parent=gene_2;gene_id=gene_2;gene=RRP7A;transcript_id=asmbl_5;cs=2
ID=asmbl_6;Parent=gene_2;gene_id=gene_2;gene=RRP7A;transcript_id=asmbl_6;cs=1
ID=asmbl_7;Parent=gene_2;gene_id=gene_2;gene=RRP7A;transcript_id=asmbl_7;cs=1
ID=asmbl_8;Parent=gene_2;gene_id=gene_2;gene=RRP7A;transcript_id=asmbl_8;cs=1
ID=asmbl_9;Parent=gene_2;gene_id=gene_2;gene=RRP7A;transcript_id=asmbl_9;cs=1
ID=asmbl_10;Parent=gene_2;gene_id=gene_2;gene=RRP7A;transcript_id=asmbl_10;cs=1


In [22]:
mrna_anno = anno = pd.read_csv('/data/isshamie/genome/mRNA_updated_final_sort.gff3',sep='\t',header=None)

In [24]:
txns = mrna_anno[8].str.extract('transcript_id=asmbl_(.+?);')#.group(1)

/home/isshamie/software/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [27]:
mrna_anno['transcript_id'] = txns

In [23]:
mrna_anno.loc[0,8]

'ID=asmbl_1;Parent=gene_1_1;gene_id=gene_1_1;gene=gene_1;transcript_id=asmbl_1;cs=5'

In [28]:
mrna_anno

,0,1,2,3,4,5,6,7,8,transcript_id
0,picr_0,merge,mRNA,10,41421,.,+,.,ID=asmbl_1;Parent=gene_1_1;gene_id=gene_1_1;ge...,1
1,picr_0,merge,mRNA,176,38694,.,+,.,ID=asmbl_2;Parent=gene_1_2;gene_id=gene_1_2;ge...,2
2,picr_0,merge,mRNA,3880,39299,.,+,.,ID=asmbl_3;Parent=gene_1_3;gene_id=gene_1_3;ge...,3
3,picr_0,merge,mRNA,7715,46630,.,-,.,ID=asmbl_4;Parent=gene_2;gene_id=gene_2;gene=R...,4
4,picr_0,merge,mRNA,7715,44542,.,-,.,ID=asmbl_5;Parent=gene_2;gene_id=gene_2;gene=R...,5
5,picr_0,merge,mRNA,7715,45912,.,-,.,ID=asmbl_6;Parent=gene_2;gene_id=gene_2;gene=R...,6
6,picr_0,merge,mRNA,7759,44597,.,-,.,ID=asmbl_7;Parent=gene_2;gene_id=gene_2;gene=R...,7
7,picr_0,merge,mRNA,7767,44597,.,-,.,ID=asmbl_8;Parent=gene_2;gene_id=gene_2;gene=R...,8
8,picr_0,merge,mRNA,7789,44568,.,-,.,ID=asmbl_9;Parent=gene_2;gene_id=gene_2;gene=R...,9
9,picr_0,merge,mRNA,7821,44513,.,-,.,ID=asmbl_10;Parent=gene_2;gene_id=gene_2;gene=...,10


In [33]:
mrna_anno[mrna_anno['transcript_id'] == '100774283']

,0,1,2,3,4,5,6,7,8,transcript_id


In [32]:
mrna_anno[mrna_anno['transcript_id'] == '500']

,0,1,2,3,4,5,6,7,8,transcript_id
472,picr_0,merge,mRNA,12881353,12917255,.,-,.,ID=asmbl_500;Parent=gene_152;gene_id=gene_152;...,500


In [38]:
mrna_anno[mrna_anno['transcript_id'] == '86760']

,0,1,2,3,4,5,6,7,8,transcript_id
77819,picr_99,merge,mRNA,3795361,3933613,.,-,.,ID=asmbl_86760;Parent=gene_30665;gene_id=gene_...,86760


## Get genes that are in the new genome

In [ ]:
unique_human_names = sec
secM_in_